In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from datetime import timedelta
from timeit import default_timer as timer
from math import sqrt

## Images reading&resizing

In [ ]:
img_l = np.asanyarray(Image.open("hangerL-small.png").convert("RGB")).astype(np.int)
img_r = np.asanyarray(Image.open("hangerR-small.png").convert("RGB")).astype(np.int)

In [ ]:
#img_l = img_l[:-10]
#img_r = img_r[10:]

In [ ]:
img_l.shape

In [ ]:
img_r.shape

## Loss functions definition

In [ ]:
def g_loss_l1(a, b, par=0):
    return abs(a) + abs(b)

def g_loss_l2(a, b, par=0):
    return sqrt(abs(a)**2 + abs(b)**2)

def g_loss_min_l1(a, b, par):
    return min(par, abs(a) + abs(b))

def g_loss_min_l2(a, b, par):
    return min(par, sqrt(abs(a)**2 + abs(b)**2))

In [ ]:
def f_loss_l1(a):
    return np.sum(np.abs(a), axis=-1)

def f_loss_l2(a):
    return np.sqrt(np.sum(a**2, axis=-1))

## Params definition

In [ ]:
kg_max=30
kv_max=40

kv_half = kv_max//2

alpha=40
g_loss = [g_loss_l1, g_loss_l2, g_loss_min_l1, g_loss_min_l2][3]
f_loss = [f_loss_l1, f_loss_l2][1]
b = 20

## Precomputing node2node losses

In [ ]:
height = img_r.shape[0]
length = img_r.shape[1]

g = np.zeros((kg_max*kv_max, kg_max*kv_max), dtype=np.float32)

for k1 in range(kg_max*kv_max):
    for k2 in range(kg_max*kv_max):
        kg1 = k1 % kg_max
        kv1 = k1 // kg_max
        kg2 = k2 % kg_max
        kv2 = k2 // kg_max
        g[k1,k2] = g_loss(kg1 - kg2,kv1 - kv2, b)
g = alpha*g

In [ ]:
kg1

In [ ]:
kv1

## Minimal path finding and recording

In [ ]:
start = timer()
img_shift = np.ones((height, length))

for i in range(img_shift.shape[0]):
    #start = timer()
    f = np.inf*np.ones((length, kg_max*kv_max), dtype=np.float32)
    for k in range(1, kg_max*kv_max):
        kg1 = k % kg_max
        kv1 = k // kg_max - kv_half
        if i+kv1 < height and i+kv1 >= 0:
            f[:length-kg1, k] = f_loss(img_r[i,:length-kg1] - img_l[i+kv1,kg1:])

    #end = timer()
    #print('\n\nconstruct graph:',timedelta(seconds=end-start))            
    #start = timer()

    pass_to_prev = np.zeros(f.shape, dtype=np.int)
    for p in range(1, length):
        ta = f[p-1, :] + g
        ind = np.argmin(ta, axis=1)
        f[p, :] += np.min(ta, axis=1)
        pass_to_prev[p, :] = ind
    
    #end = timer()
    #print('\n\nfind path:',timedelta(seconds=end-start))
    #start = timer()
    
    line_shift = np.ones(length, dtype=np.int)
    line_shift[-1] = np.argmin(f[-1,:])
    for p in reversed(range(length-1)):
        line_shift[p] = pass_to_prev[p+1, line_shift[p+1]]
    
    img_shift[i] = line_shift
    #end = timer()
    #print('\n\nrecover path:',timedelta(seconds=end-start)) 

    print(i, end=" ")
    #print("\n"+20*"=")

end = timer()
print('\n\n\nTime per image',timedelta(seconds=end-start))

## output image transforming and shifts decoding

In [ ]:
img_shift2 = np.zeros((height, length, 3), dtype=np.int)
img_shift2[:,:,0] = img_shift % kg_max # horizontal
img_shift2[:,:,1] = img_shift // kg_max - kv_half # vertical

np.save("img_shift.npy", img_shift2)

img_shift3 = img_shift2.copy()

img_shift3 = np.linalg.norm(img_shift3, axis=-1)
img_shift3 = 255*img_shift3/img_shift3.max()

#img_shift2 = 255*img_shift2/img_shift2.max()
#img_shift2[:,:,0] = 255*img_shift2[:,:,0]/img_shift2[:,:,0].max()
#img_shift2[:,:,1] = 255*img_shift2[:,:,1]/img_shift2[:,:,1].max()

In [ ]:
img_shift2[:,:,0].max()

## Horizontal shift (normed at 255)

In [ ]:
Image.fromarray((255*img_shift2[:,:,0]/img_shift2[:,:,0].max()).astype(np.uint8)).resize((800,600), Image.BICUBIC)

## Vertical shift (normed at 255)

In [ ]:
Image.fromarray((255*img_shift2[:,:,1]/img_shift2[:,:,1].max()).astype(np.uint8)).resize((800,600), Image.BICUBIC)

## Color shift image (normed at 255)

In [ ]:
Image.fromarray(img_shift2.astype(np.uint8)).resize((800,600), Image.BICUBIC)

## Norm maps (normed at 255)

In [ ]:
Image.fromarray(img_shift3.astype(np.uint8)).resize((800,600), Image.BICUBIC)

## Original image

In [ ]:
Image.fromarray(img_l.astype(np.uint8)).resize((800,600))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(img_shift2[:,:,1].reshape(-1))